In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
df = pd.read_csv('v_data.csv')

In [3]:
#This is a pruned dataframe with a subset of the columns that we'll be using for this exploratory anaylsis
values = ['total_vaccinations', 'people_vaccinated', 'daily_vaccinations_per_million']
prn_df = df[['location', 'date', 'total_vaccinations', 'people_vaccinated', 'daily_vaccinations_per_million']]

In [4]:
#This is to see which locations have the most missing data, we could have ran count() on any of the columns 
#since dropna() drops all Na rows
count_series = prn_df.dropna().groupby('location')['date'].count()

In [5]:
#describe tells us the average number of entries is 111, but std-dev is fairly significant at 93, we'll need to look at the list 
#of indices (locations) with above average sampling, as better reporting probably implies higher quality data
count_series.describe()['mean']

111.48497854077253

In [6]:
x = count_series.describe()['mean']
ab_avg = [ind for ind in count_series.index if count_series[ind]>x]
ab_avg

['Africa',
 'Argentina',
 'Aruba',
 'Asia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belgium',
 'Bolivia',
 'Brazil',
 'Bulgaria',
 'Cambodia',
 'Canada',
 'Chile',
 'Colombia',
 "Cote d'Ivoire",
 'Croatia',
 'Curacao',
 'Czechia',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'England',
 'Estonia',
 'Europe',
 'European Union',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Gibraltar',
 'Greece',
 'Guatemala',
 'High income',
 'Hong Kong',
 'Hungary',
 'India',
 'Indonesia',
 'Ireland',
 'Isle of Man',
 'Israel',
 'Italy',
 'Japan',
 'Kazakhstan',
 'Kenya',
 'Latvia',
 'Lebanon',
 'Liechtenstein',
 'Lithuania',
 'Low income',
 'Lower middle income',
 'Luxembourg',
 'Macao',
 'Malawi',
 'Malaysia',
 'Maldives',
 'Malta',
 'Mexico',
 'Moldova',
 'Mongolia',
 'Montenegro',
 'Morocco',
 'Namibia',
 'Nepal',
 'New Zealand',
 'North America',
 'Northern Ireland',
 'Norway',
 'Oceania',
 'Palestine',
 'Panama',
 'Paraguay',
 'Peru',
 'Poland',
 '

In [7]:
#List seems fairly varied, we'll be arbitrarily selecting some countries to include in our analyses
Countries = ['World', 'Asia', 'Africa', 'Europe', 
             'France', 'Italy', 'Spain', 'Germany', 'United Kingdom', 'Russia', 'Scotland', 'Norway', 'Belgium',
             'United States', 'Canada', 'Mexico', 'Brazil', 
             'Zimbabwe', 'South Africa', 'Morocco', 'Algeria', 'Tunisia',
             'Taiwan', 'Hong Kong', 'India', 'Thailand', 'Sri Lanka', 'Japan', 'South Korea', 'Vietnam', 'Indonesia', 'Australia',
             'Qatar', 'Bahrain', 'Israel', 'Turkey',
             'Low income', 'High income', 'Upper middle income', 'Lower middle income' 
            ]
#final_df = prn_df.dropna().drop(index=[prn_df[ind].index for ind in list((set(count_series.index)-set(Countries)))])

In [8]:
#Computing a final dataframe 
final_df = prn_df.pivot_table(values, index=['location','date']).loc[Countries]

In [9]:
final_df.fillna(0, inplace=True)

In [11]:
final_df.dtypes

daily_vaccinations_per_million    float64
people_vaccinated                 float64
total_vaccinations                float64
dtype: object